# Task 1: Prompt Chaining for Customer Support AI

## Overview
This notebook demonstrates **prompt chaining**, a technique where complex tasks are broken down into sequential steps. Each step's output feeds into the next, creating a chain of prompts that work together.

## Use Case
We'll build a customer support system with 4 chained steps:
1. **Intent Classification** - Identify the issue type
2. **Information Gathering** - Extract key details
3. **Solution Generation** - Create a solution
4. **Response Formatting** - Format as professional response

In [1]:
# Import required libraries
import os
from google import genai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the Gemini client
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

print("✓ Gemini client initialized successfully")

✓ Gemini client initialized successfully


## Sample Customer Message
Let's define a customer support query to process through our chain:

In [2]:
customer_message = """
Hi, I've been trying to log into my account for the past hour but keep getting an error message 
saying 'Invalid credentials'. I'm sure my password is correct because I saved it in my password 
manager. I tried resetting it twice but I'm not receiving the reset emails. This is really 
frustrating because I need to access my account urgently for an important project deadline.
"""

print("Customer Message:")
print(customer_message)

Customer Message:

Hi, I've been trying to log into my account for the past hour but keep getting an error message 
saying 'Invalid credentials'. I'm sure my password is correct because I saved it in my password 
manager. I tried resetting it twice but I'm not receiving the reset emails. This is really 
frustrating because I need to access my account urgently for an important project deadline.



## Step 1: Intent Classification
First, we classify the customer's intent to understand what type of issue they're facing.

In [3]:
# Prompt 1: Classify the intent
intent_prompt = f"""
You are a customer support intent classifier. Analyze the following customer message and classify it into one of these categories: Technical Issue, Billing Question, Feature Request, Account Access, or General Inquiry.

Customer Message: {customer_message}

Respond with ONLY the category name.
"""

response_1 = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=intent_prompt
)

intent = response_1.text.strip()
print(f"Step 1 - Intent Classification: {intent}")
print("-" * 60)

Step 1 - Intent Classification: Account Access
------------------------------------------------------------


## Step 2: Information Gathering
Extract key details and identify missing information needed to solve the issue.

In [4]:
# Prompt 2: Extract information
info_prompt = f"""
Based on the customer message below and its classified intent, extract the key information needed to resolve this issue. List the specific details mentioned and any missing information that should be asked.

Intent: {intent}
Customer Message: {customer_message}

Provide a structured summary of:
1. Key details mentioned
2. Missing information needed
"""

response_2 = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=info_prompt
)

extracted_info = response_2.text
print("Step 2 - Information Gathering:")
print(extracted_info)
print("-" * 60)

Step 2 - Information Gathering:
Here's a structured summary of the key details and missing information, based on the customer message and intent:

**1. Key Details Mentioned:**

*   **Problem:** Unable to log into account.
*   **Error Message:** "Invalid credentials"
*   **Password Certainty:** Believes password is correct because it's saved in a password manager.
*   **Troubleshooting Steps Taken:** Attempted password reset twice.
*   **Problem with Troubleshooting:** Not receiving password reset emails.
*   **Urgency:** Needs access urgently due to an important project deadline.

**2. Missing Information Needed:**

*   **Username or Email Address:** What username or email address is the customer using to log in? (This is the most crucial piece of missing information)
*   **Account Type/Platform:** What platform or service is the customer trying to access (e.g., website name, app name)? Knowing this ensures the correct troubleshooting steps are taken.
*   **Account Creation Date:** Wh

## Step 3: Solution Generation
Generate a solution based on the classified intent and extracted information.

In [5]:
# Prompt 3: Generate solution
solution_prompt = f"""
Generate a solution for this customer support case:

Intent: {intent}
Customer Message: {customer_message}
Key Information: {extracted_info}

Provide a clear, step-by-step solution or answer to address the customer's needs.
"""

response_3 = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=solution_prompt
)

solution = response_3.text
print("Step 3 - Solution Generation:")
print(solution)
print("-" * 60)

Step 3 - Solution Generation:
Okay, here's a solution to address the customer's account access issue, broken down into steps:

**Step 1: Initial Response and Empathy**

> "Hi there, I understand how frustrating it is to be locked out of your account, especially with an urgent project deadline. I'm sorry you're experiencing this. I'll do my best to help you regain access as quickly as possible."

**Step 2: Gather Missing Information (Politely and Efficiently)**

> "To help me troubleshoot this, could you please provide the following information?
> *   **Your username or email address** associated with the account.
> *   **The name of the platform or service** you're trying to access (e.g., website name, app name).
> *   **Approximately when you created the account**?
> *   **What time did you attempt the password resets?**"

**Step 3: Address Potential Email Delivery Issues (While Waiting for Information)**

> "While I wait for that information, let's check a few common causes for missi

## Step 4: Response Formatting
Format the solution as a professional, empathetic customer service response.

In [6]:
# Prompt 4: Format response
format_prompt = f"""
Format the following solution as a professional, empathetic customer support response:

Customer Issue: {customer_message}
Solution: {solution}

Create a complete response that:
- Acknowledges the customer's concern
- Provides the solution clearly
- Offers additional help
- Maintains a professional and friendly tone
"""

response_4 = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=format_prompt
)

final_response = response_4.text
print("Step 4 - Final Customer Response:")
print("=" * 60)
print(final_response)
print("=" * 60)

Step 4 - Final Customer Response:
Subject: Regarding your account access issue

Hi [Customer Name, if available, otherwise leave blank],

I understand how frustrating it is to be locked out of your account, especially with an urgent project deadline. I'm truly sorry you're experiencing this and I want to assure you I'll do my best to help you regain access as quickly as possible.

First, let's try to pinpoint what might be happening. Could you please provide me with the following information? This will help me investigate further:

*   **Your username or email address** associated with the account.
*   **The name of the platform or service** you're trying to access (e.g., website name, app name).
*   **Approximately when you created the account**? (Month/Year is sufficient).
*   **What time did you attempt the password resets?** (Approximately).

While I wait for that information, let's check a few common causes for missing password reset emails:

*   **Check your spam/junk folder:** S

## Summary: The Complete Chain
Let's visualize the entire prompt chain:

In [7]:
print("\n📋 PROMPT CHAIN SUMMARY\n")
print(f"Input: Customer Message\n   → {customer_message[:80]}...\n")
print(f"Chain Step 1: Intent Classification\n   → {intent}\n")
print(f"Chain Step 2: Information Gathering\n   → Extracted key details and missing info\n")
print(f"Chain Step 3: Solution Generation\n   → Created step-by-step solution\n")
print(f"Chain Step 4: Response Formatting\n   → Professional customer response\n")
print("✓ Prompt chain completed successfully!")


📋 PROMPT CHAIN SUMMARY

Input: Customer Message
   → 
Hi, I've been trying to log into my account for the past hour but keep getting ...

Chain Step 1: Intent Classification
   → Account Access

Chain Step 2: Information Gathering
   → Extracted key details and missing info

Chain Step 3: Solution Generation
   → Created step-by-step solution

Chain Step 4: Response Formatting
   → Professional customer response

✓ Prompt chain completed successfully!


## Key Takeaways

**Prompt Chaining Benefits:**
1. **Modularity** - Each step handles a specific task
2. **Transparency** - Easy to debug and understand each step
3. **Flexibility** - Steps can be modified independently
4. **Reliability** - Breaking complex tasks into smaller pieces improves accuracy

**When to Use Prompt Chaining:**
- Complex multi-step workflows
- Tasks requiring different types of reasoning
- When intermediate outputs need validation
- Building maintainable AI systems